In [1]:
import numpy as np
import math
import random
from PIL import Image 
import matplotlib.pyplot as plt
from matplotlib import image
import cv2
import seaborn as sns

## Graph-cut based segmentation using Pymaxflow
Using energy minimization alogorithms such as aexpansion_grid and abswap_grid

In [2]:
from maxflow.fastmin import aexpansion_grid, abswap_grid, energy_of_grid_labeling

def graph_cut(img,type_,level_start):

    img = cv2.GaussianBlur(img,(5,5),0)
    d=25;s=55;
    img = cv2.bilateralFilter(img, d, s, s)

    I = img
    I = I[:,:,1]/I.max()
    #print("Img shape: ", img.shape)
    #print("I shape: ", I.shape)

    # Generates 16 gray levels for nearsest prototype labeling
    L = 2
    levs = np.arange(level_start/L, 1, 1/L) # 0.92 for 1 and 3
    #print(levs)
    #print("levs: ", levs)

    # Calculate data cost as the absolute difference between the label prototype and the pixel value
    D = np.abs(I.reshape(I.shape+(1,)) - levs.reshape((1,1,-1)))
    #print("I.reshape(I.shape+(1,)): ", I.reshape(I.shape+(1,)))
    #print("levs.reshape((1,1,-1): ",levs.reshape((1,1,-1)))
    #print("I.reshape(I.shape+(1,)) - levs.reshape((1,1,-1)): ", I.reshape(I.shape+(1,)) - levs.reshape((1,1,-1)))
    #print("D: ", D)
    
    # Generate nearest prototype labeling
    Id = np.argmin(D,2)
    #print("Id: ", Id)

    # Calculate neighbourhood cost as absolute difference between prototypes 
    alpha = 1
    V = alpha * np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1)))
    #print("V: ", V)
    
    # Mimimise data + neighbourhood cost
    if type_== "abswap_grid":
        labels_gc = abswap_grid(D,V,max_cycles=1000)
    elif type_ == "aexpansion_grid":
        labels_gc = aexpansion_grid(D,V,max_cycles=1000)

    #energy_of_grid_labeling(D,V,labels=2)
    # abswap_grid aexpansion_grid

    '''
    plt.figure(figsize = (20,6))
    plt.subplot(1,2,1)
    plt.imshow(Id, cmap = 'gray')
    plt.title(label = 'Direct labeling')

    plt.subplot(1,2,2)
    plt.imshow(labels, cmap = 'gray')
    plt.title(label = 'Regularised labeling')
    plt.show
    '''
    #plt.imshow(labels5_gc, cmap = 'gray')
    return labels_gc


In [3]:
def binarize(img,Thres_blockSize, thres_switch):
    
    thres= img
    ###Assigning values based on Tracker’s position
    thres_switch = cv2.getTrackbarPos('thres_switch','trackbar')
    #Assign Kernel blocksize for Thresholding only if trackbar is active

    Thres_blockSize = 1 + 2*cv2.getTrackbarPos('Thres_blockSize','trackbar')
    #Assign Shift contstant value for Thresholding only if trackbar is active
    if cv2.getTrackbarPos('c_sub','trackbar'):
        c_sub = cv2.getTrackbarPos('c_sub','trackbar')
    elif thres_switch == 1:
        thres = cv2.adaptiveThreshold (img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, Thres_blockSize, c_sub)

    #Binarizing  using Thresold_Otsu after a Gausian blur if thres_otsu variable is in ON position  
    elif thres_switch == 2:
        #thresh = threshold_otsu(image)
        #binary = image > thresh
        gaus = cv2.GaussianBlur(img,(3,3),0)
        ret3,thres = cv2.threshold(gaus,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
    return thres


In [4]:
def plotHist(img,showw):
    hist,bins = np.histogram(img.flatten(),256,[0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()
    plt.plot(cdf_normalized, color = 'r')
    plt.hist(img.flatten(),256,[0,256], color = 'g')
    plt.xlim([0,256])
    plt.legend(('cdf','histogram'), loc = 'lower right')
    if showw == "True":
        plt.show()
    return plt


def PSNR(image1, image2):

    # OpenCV
    #psnr =  cv2.PSNR(image1, image2)
    #print('OpenCV PSNR: ',psnr)39

    # Own implementation
    mse = np.mean((image1.astype(np.float64) / 255 - image2.astype(np.float64) / 255) ** 2)
    psnr = 10 * math.log10(1. / mse)
    #print('Own implementation: ', 10 * math.log10(1. / mse))
    return psnr


In [5]:
def confusion_mat_seg(y_true, y_pred_t, setname):
    """
    

    Parameters
    ----------
    y_true : Ground truth mask 
    y_pred_t : Predicted mask
    setname : Name description for the confusion matrix
    mydir : Outout directory to save the cm image

    Returns
    -------
    None.

    """
    tp = np.logical_and(y_true==True, y_pred_t==True)
    tn = np.logical_and(y_true==False, y_pred_t==False)
    fp = np.logical_and(y_true==True, y_pred_t==False)
    fn = np.logical_and(y_true==False, y_pred_t==True)
    
    cmat = [[np.sum(tp), np.sum(fn)], [np.sum(fp), np.sum(tn)]]


    plt.figure(figsize = (6,6))
    plt.title(setname)
    sns.heatmap(cmat/np.sum(cmat), cmap="Reds", annot=True, fmt = '.2%', square=1, linewidth=2.)
    plt.xlabel("predictions")
    plt.ylabel("real values")
    #plt.savefig(mydir + setname + 'confusion_mat_seg.png')
    try:
        plt.show()
    except:
        pass
    
    iou = np.sum(tp)/(np.sum(tp)+np.sum(fn)+np.sum(fp))
    f1 = (2*np.sum(tp))/((2*np.sum(tp))+np.sum(fn)+np.sum(fp))
    precision =  np.sum(tp)/(np.sum(tp)+np.sum(fp)) #pixcel_accuracy
    
    acc = (np.sum(tp)+np.sum(tn))/(np.sum(tp)+np.sum(tn)+np.sum(fn)+np.sum(fp))
    sensitivity= np.sum(tp)/(np.sum(tp)+np.sum(fn)) 
    specificity = np.sum(tn)/(np.sum(tn)+np.sum(fp)) 
    
    print('IoU score: ',iou,'\nF1 score:', f1,'\nPrecision:', precision,'\nSensitivity:', sensitivity,'\nSpecificity:', specificity,'\nAccuracy:', acc)
    return([iou, f1, precision, sensitivity, specificity, acc])
    